# Create a face mask
- Since we found that our blendshapes so far were including the deformation (yellow region shown below) back of head, we need to somehow take into consideration only vertices which effectively creates facial expressions

![blenshape_withBackOfheadDeformation](../images/Blendshape_withBackOfHeadDeformation.png)


- In order to do that, we need to obtain a face mask (vertex id lists) which represents the corresponding face mask regions as below

![face_mask_region](../images/Facemask_side01.png)



# Method
- We employ nearest neighboring search since it was hard to create a face mask which has isotropic distribution from the top of nose by selectinig vertices by a hand. (This approach requires additional setup to visualize mesh and interactive i/o)

- In order to get a face mask which equally cover the face from the top of nose in all directions, we use nearest neighboring search with using KD-tree. By using KD-tree, we can efficienlty search nearest neighboring vertices of the certain point which given to the query

# Tools
- Scikit-learn KDTree

## import packages and utils

In [ ]:
import numpy as np
import os
from sklearn.neighbors import KDTree
from utils.OBJ_helper import OBJ
import trimesh

data_type = "3dgs"
# data_type = "tracked_mesh"

# Gaussian splatting
if data_type == "3dgs":
    ply_name = "sample_subd2_face"
    nearestNeighbors = 50000
    # ply_name = "sample_face"
    ply_fname = ply_name + ".ply"
    path_to_samples =  os.path.join(os.getcwd(), "samples", "3dgs")
    path_to_mesh =os.path.join(path_to_samples, ply_fname)
    save_path = path_to_samples
    assert os.path.exists(path_to_mesh)
elif data_type == "tracked_mesh":
    # tracked mesh
    obj_name = "sample"
    nearestNeighbors = 3000
    obj_fname = obj_name + ".obj"
    path_to_samples =  os.path.join(os.getcwd(), "samples")
    path_to_mesh =os.path.join(path_to_samples, obj_fname)
    save_path = path_to_samples
    assert os.path.exists(path_to_mesh)


set path to mesh which is given as a sample for the vertices selection and save_path

In [ ]:
# # original mesh
# mesh_loader = "original"
# averageMesh_obj = OBJ(path_to_mesh, swapyz=False)
# # Select the vertex which will feed to query of nearest neighboring search
# # - we select the vertex which represents the top of nose
# c_id =3567

In [ ]:
# trimesh
mesh_loader = "trimesh"
averageMesh_obj = trimesh.load(path_to_mesh, force = 'mesh')
# Select the vertex which will feed to query of nearest neighboring search
# - we select the vertex which represents the top of nose
c_id =2658

In [ ]:
num_vertices = len(averageMesh_obj.vertices)
len_col = num_vertices*3

In [ ]:
array_X = np.array(averageMesh_obj.vertices)
X = np.array(averageMesh_obj.vertices).reshape(len(averageMesh_obj.vertices), 3)
print(X.shape)

Perform KDTree nearest neighboring vertex search
- ind contains the indices list of the selected regions from the top nose

In [ ]:
tree = KDTree(X, leaf_size=100, metric='euclidean')
dist, ind = tree.query(X[c_id:c_id+1], k = nearestNeighbors) # ind: indices of k closest neighbors

Convert the array of index to the list

In [ ]:
selected_vertices = []
for id in ind:
    selected_vertices.append(X[id])

selected_vertices = np.array(selected_vertices)
selected_vertices = selected_vertices.squeeze()
print(selected_vertices)

vertices_list = []
for x, y, z in selected_vertices:
    vertices_list.append(x)
    vertices_list.append(y)
    vertices_list.append(z)

vertices_list = np.array(vertices_list)
print(vertices_list)


write a obj file to visualize the pointclouds

In [ ]:
if data_type == "3dgs":
    OBJ.write_PointClouds(save_path = save_path, vertices = vertices_list, mesh_name = "FaceMask_" +ply_name+"_" + mesh_loader)
elif data_type == "tracked_mesh":
    OBJ.write_PointClouds(save_path = save_path, vertices = vertices_list, mesh_name = "FaceMask_" +obj_name+"_" + mesh_loader)

list of the indices

In [ ]:
list_facemask_ids = []
for id in ind: # ind = 5000 nearest neighboring vertex ids of #3567
    list_facemask_ids=id
print(list_facemask_ids)

Create a bit mask of the vertices

In [ ]:
bit_mask = np.zeros(len_col, dtype = int)
for id in list_facemask_ids:
    bit_mask[3*id] = int(1)
    bit_mask[3*id+1] = int(1)
    bit_mask[3*id+2] = int(1)

print(bit_mask)
print(bit_mask.shape)

In [ ]:
from utils.converter import vector2MatNx3
matNx3_bitMask = vector2MatNx3(bit_mask, num_vertices)
print(matNx3_bitMask.shape)

Use dataclass to serialize the index list of the face mask

In [ ]:
from dataclasses import dataclass
from utils.Blendshape import FaceMask

In [ ]:
facemask = FaceMask(center_id=c_id, list_ids=list_facemask_ids, bit_mask = matNx3_bitMask)

In [ ]:
print(facemask.center_id)
print(facemask.list_ids)

In [ ]:
from datetime import datetime
# get current date and year
now = datetime.now()

date = now.strftime("%d") + now.strftime("%m") + now.strftime("%Y")
print(date)
time = now.strftime("%H_%M")
print("time:", time)

dump instance of FaceMask

In [ ]:
from utils.pickel_io import dump_pckl, load_from_memory
import os
if data_type == "tracked_mesh":
    pickel_fname = "FaceMask_"+obj_name+"_"+date+"_"+time+"_"+mesh_loader+".pkl"
elif data_type == "3dgs":
    pickel_fname = "FaceMask_"+ply_name+"_"+date+"_"+time+"_"+mesh_loader+".pkl"
dump_pckl(data=facemask, save_root=save_path, pickel_fname=pickel_fname)

load instance of FaceMask to check if we could store the data

In [ ]:
load_from_memory(path_to_memory=save_path, pickle_fname=pickel_fname)